In [6]:
# All Required imports
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

In [51]:
train_data = pd.read_csv('train.csv')
train_data = train_data.dropna()
train_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [57]:
test_df = train_data.drop(['Ticket','Name','PassengerId','Cabin'], axis=1)

In [58]:
test_df['Female'] = test_df['Sex'] == 'female'

In [59]:
test_df = test_df.drop('Sex', axis=1)

In [60]:
test_df['Embarked'] = test_df['Embarked'] == 'S'

In [61]:
test_df

,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked,Female
1,1,1,38.0,1,0,71.2833,False,True
3,1,1,35.0,1,0,53.1000,True,True
6,0,1,54.0,0,0,51.8625,True,False
10,1,3,4.0,1,1,16.7000,True,True
11,1,1,58.0,0,0,26.5500,True,True
...,...,...,...,...,...,...,...,...
871,1,1,47.0,1,1,52.5542,True,True
872,0,1,33.0,0,0,5.0000,True,False
879,1,1,56.0,0,1,83.1583,False,True
887,1,1,19.0,0,0,30.0000,True,True
